# Eli Lilly Data Engineer Task

1. Create a query that calculates the average number of goals per game between 1900 and 2000.
2. Create a query that counts the number of shootouts wins by country and arrange in alphabetical order.
3. Create a reliable key that allows the joining together of goal scorers, results, and shootouts.
4. Create a query that identifies which teams have won a penalty shootout after a 1-1 draw.
5. Create a query that identifies the top goal scorer by tournament, and what percentage that equates to for all goals scored in the tournament.

6. Create an additional column that flags records with data quality issues.
7. Resolve the identified quality issues.

Start by importing the csv files using Pandas.

In [303]:
import pandas as pd
import numpy as np

goalscorers_df = pd.read_csv('goalscorers.csv')
results_df = pd.read_csv('results.csv')
shootouts_df = pd.read_csv('shootouts.csv')

## Task 1

The results.csv file contains data on number of goals per game in each year.

In [304]:
results_df.head(5)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [305]:
results_df.dtypes

date          object
home_team     object
away_team     object
home_score     int64
away_score     int64
tournament    object
city          object
country       object
neutral         bool
dtype: object

We see how data is an object, would be best to convert to datetime format so we can extract the year.

In [306]:
results_df['date'] = pd.to_datetime(results_df['date'])

We now filter the results where the game was played between 1900 and 2000.

In [307]:
filtered_results = results_df[(results_df['date'].dt.year >= 1900) & (results_df['date'].dt.year < 2000)]

We sum all the home goals and away goals to find the total goals scored per game.

In [308]:
total_goals = filtered_results['home_score'].sum() + filtered_results['away_score'].sum()

We now find how many games were played between these years, in order to find the average number of goals per game.

In [309]:
number_games = len(filtered_results)

average_goals = total_goals / number_games

average_goals

3.0824074074074073

## Task 2

The shootouts_df DataFrame contains the results of all the penalty shootouts.

In [310]:
shootouts_df.head(5)

,date,home_team,away_team,winner,first_shooter
0,1967-08-22,India,Taiwan,Taiwan,NaN
1,1971-11-14,South Korea,Vietnam Republic,South Korea,NaN
2,1972-05-07,South Korea,Iraq,Iraq,NaN
3,1972-05-17,Thailand,South Korea,South Korea,NaN
4,1972-05-19,Thailand,Cambodia,Thailand,NaN


We can group by each winning team and find the number of times this team appears as a winner using .size(). .sort_index() sorts  by alphabetical order. We then turn this into a pandas DataFrame and rename the column to count.

In [311]:
penalty_winner = pd.DataFrame(shootouts_df.groupby('winner').size().sort_index())
penalty_winner = penalty_winner.rename(columns={0: 'count'})

penalty_winner

,count
winner,
Abkhazia,2
Algeria,7
Angola,7
Antigua and Barbuda,2
Argentina,14
...,...
Zambia,13
Zanzibar,2
Zimbabwe,7


## Task 3

We start by identifying common columns in each dataset.

In [312]:
shootouts_df.columns, results_df.columns, goalscorers_df.columns

(Index(['date', 'home_team', 'away_team', 'winner', 'first_shooter'], dtype='object'),
 Index(['date', 'home_team', 'away_team', 'home_score', 'away_score',
        'tournament', 'city', 'country', 'neutral'],
       dtype='object'),
 Index(['date', 'home_team', 'away_team', 'team', 'scorer', 'minute',
        'own_goal', 'penalty'],
       dtype='object'))

'date', 'home_team', and 'away_team' are common to all datasets, allowing us to map each game uniquely. We can create composite keys with these columns.

In [313]:
shootouts_df['match_key'] = (
    shootouts_df['date'].astype(str) + "_" + shootouts_df['home_team'] + "_" + shootouts_df['away_team']
)
results_df['match_key'] = (
    results_df['date'].astype(str) + "_" + results_df['home_team'] + "_" + results_df['away_team']
)
goalscorers_df['match_key'] = (
    goalscorers_df['date'].astype(str) + "_" + goalscorers_df['home_team'] + "_" + goalscorers_df['away_team']
)

## Task 4

We want to merge the shootouts dataset with the results dataset and find all the games where the #away goals = #home goals = 1. We use an inner join rather than a left join because there appears to be data for a shootout that doesnt appear in the results table.

In [314]:
merged_data = shootouts_df.merge(results_df, on='match_key', how='inner')

merged_data_left = shootouts_df.merge(results_df, on='match_key', how='left')

difference = merged_data_left[~merged_data_left['match_key'].isin(merged_data['match_key'])]

difference #Saare County vs Aland Islands 2011 match is not present in results.csv

,date_x,home_team_x,away_team_x,winner,first_shooter,match_key,date_y,home_team_y,away_team_y,home_score,away_score,tournament,city,country,neutral
378,2011-06-29,Saare County,Åland Islands,Åland Islands,NaN,2011-06-29_Saare County_Åland Islands,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Filter by games where the home team scored just once, and the away team scored just once.

In [315]:
filtered_results = merged_data[(merged_data['home_score'] == 1) & (merged_data['away_score'] == 1)]
filtered_results.head()

,date_x,home_team_x,away_team_x,winner,first_shooter,match_key,date_y,home_team_y,away_team_y,home_score,away_score,tournament,city,country,neutral
0,1967-08-22,India,Taiwan,Taiwan,NaN,1967-08-22_India_Taiwan,1967-08-22,India,Taiwan,1,1,Merdeka Tournament,Kuala Lumpur,Malaysia,True
1,1971-11-14,South Korea,Vietnam Republic,South Korea,NaN,1971-11-14_South Korea_Vietnam Republic,1971-11-14,South Korea,Vietnam Republic,1,1,King's Cup,Bangkok,Thailand,True
3,1972-05-17,Thailand,South Korea,South Korea,NaN,1972-05-17_Thailand_South Korea,1972-05-17,Thailand,South Korea,1,1,AFC Asian Cup,Bangkok,Thailand,False
6,1973-06-14,Guinea,Mali,Guinea,NaN,1973-06-14_Guinea_Mali,1973-06-14,Guinea,Mali,1,1,African Cup of Nations qualification,Conakry,Guinea,False
7,1973-06-14,Mauritius,Tanzania,Mauritius,NaN,1973-06-14_Mauritius_Tanzania,1973-06-14,Mauritius,Tanzania,1,1,African Cup of Nations qualification,Port Louis,Mauritius,False


Look at just the winner column and count how many times this team appears.

In [316]:
filtered_results['winner'].value_counts()

South Korea            7
Uruguay                7
Burkina Faso           6
Argentina              6
Mali                   5
                      ..
Antigua and Barbuda    1
Zambia                 1
Cuba                   1
Sudan                  1
Åland                  1
Name: winner, Length: 115, dtype: int64

## Task 5

We start by merging the goalscorers data and the results data on the match key and by a left join. We then group by the tournament and scorer and calculate the size using .size().

In [317]:
merged_data = goalscorers_df.merge(results_df, on='match_key', how='left')
tournament_goalscorers = merged_data.groupby(['tournament', 'scorer']).size().reset_index(name='goals')

The merged_data is used to work out the total number of goals per tournament. We did a left join with the goalscorers dataset, so every row corresponds to a goal.

In [318]:
total_goals_tournament = merged_data.groupby('tournament').size().reset_index(name='total_goals')

We then merge these two datasets on the 'tournament' column to collect information on total goals and goals per player. The percentage is calculated and the top scorer is identified.

In [319]:
tournament_goals = tournament_goalscorers.merge(total_goals_tournament, on='tournament')

tournament_goals['goal_percentage'] = (tournament_goals['goals'] / tournament_goals['total_goals']) * 100

top_scorers = tournament_goals.loc[tournament_goals.groupby('tournament')['goals'].idxmax()]

In [320]:
top_scorers

,tournament,scorer,goals,total_goals,goal_percentage
33,AFC Asian Cup,Ali Daei,14,989,1.415571
1314,African Cup of Nations,Samuel Eto'o,18,1767,1.018676
1601,Baltic Cup,Ēriks Pētersons,9,229,3.930131
1611,British Home Championship,Geoff Hurst,4,33,12.121212
1623,CONMEBOL–UEFA Cup of Champions,Claudio Caniggia,1,7,14.285714
1675,Confederations Cup,Cuauhtémoc Blanco,9,423,2.127660
2687,Copa América,Norberto Doroteo Méndez,17,2671,0.636466
3932,FIFA World Cup,Miroslav Klose,16,2720,0.588235
5891,FIFA World Cup qualification,Carlos Ruiz,39,22738,0.171519
14396,Gold Cup,Landon Donovan,18,1097,1.640839


## Task 6

We start by defining a function flag_data_quality_issues which initiates a column of False. We then supply a check to complete, if the check happens to spot an error, the corresponding False becomes True through the |= (bitwise) operation.

In [321]:
def flag_data_quality_issues(df, checks):
    flags = pd.Series(False, index=df.index)
    for check in checks:
        flags |= check(df)
    return flags    

We now define our checks for data quality. For the shootouts dataset, we see lots of isues with the first_shooter column. For the results dataset, we call back to that issue involving Saare County and so look for games where the tournament is NaN as a flag. For goalscorers, we check for missing or impossible time data.

In [322]:
def shootouts_checks(df):
    return [
        lambda df: df['first_shooter'].isna(),
        lambda df: df['winner'].isna(),
        lambda df: df['home_team'].isna(),
        lambda df: df['away_team'].isna(),  
    ]

def results_checks(df):
    return [
        lambda df: df['home_score'].isna() | df['away_score'].isna(),
        lambda df: df['tournament'].isna(),
        lambda df: df['home_team'].isna(),
        lambda df: df['away_team'].isna(), 
        lambda df: df['tournament'].isna(),
        lambda df: df['city'].isna(),
        lambda df: df['country'].isna(), 
        lambda df: df['neutral'].isna(),
    ]

def goalscorers_checks(df):
    return [
        lambda df: df['minute'].isna(), 
        lambda df: df['home_team'].isna(),
        lambda df: df['away_team'].isna(), 
        lambda df: df['team'].isna(),
        lambda df: df['scorer'].isna(), 
        lambda df: df['own_goal'].isna(),
        lambda df: df['penalty'].isna(), 
    ]

Define a new column called data_quality_flag which applies the flagging function to the checking functions.

In [323]:
shootouts_df['data_quality_flag'] = flag_data_quality_issues(shootouts_df, shootouts_checks(shootouts_df))
results_df['data_quality_flag'] = flag_data_quality_issues(results_df, results_checks(results_df))
goalscorers_df['data_quality_flag'] = flag_data_quality_issues(goalscorers_df, goalscorers_checks(goalscorers_df))

shootouts_df.head()

,date,home_team,away_team,winner,first_shooter,match_key,data_quality_flag
0,1967-08-22,India,Taiwan,Taiwan,NaN,1967-08-22_India_Taiwan,True
1,1971-11-14,South Korea,Vietnam Republic,South Korea,NaN,1971-11-14_South Korea_Vietnam Republic,True
2,1972-05-07,South Korea,Iraq,Iraq,NaN,1972-05-07_South Korea_Iraq,True
3,1972-05-17,Thailand,South Korea,South Korea,NaN,1972-05-17_Thailand_South Korea,True
4,1972-05-19,Thailand,Cambodia,Thailand,NaN,1972-05-19_Thailand_Cambodia,True


## Task 7

Replace NaN values in first shooter column with a message saying 'Not Recorded'; could have replaced it with 0. Update the data quality flag column.

In [324]:
shootouts_df['first_shooter'].fillna('Not Recorded', inplace=True)
shootouts_df['data_quality_flag'] = flag_data_quality_issues(shootouts_df, shootouts_checks(shootouts_df))
shootouts_df[shootouts_df['data_quality_flag'] == True]

,date,home_team,away_team,winner,first_shooter,match_key,data_quality_flag


We check to see if there are any data quality issues with the results dataframe.

In [325]:
results_df['data_quality_flag'] = flag_data_quality_issues(results_df, results_checks(results_df))
results_df[results_df['data_quality_flag'] == True]

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,match_key,data_quality_flag


We look for rows with data quality issues and replace NaN values with 'Not Recorded'.

In [326]:
goalscorers_df['minute'].fillna('Not Recorded', inplace=True)
goalscorers_df['scorer'].fillna('Not Recorded', inplace=True)
goalscorers_df['own_goal'].fillna('Not Recorded', inplace=True)
goalscorers_df['penalty'].fillna('Not Recorded', inplace=True)
goalscorers_df['data_quality_flag'] = flag_data_quality_issues(goalscorers_df, goalscorers_checks(goalscorers_df))
goalscorers_df[goalscorers_df['data_quality_flag'] == True]

,date,home_team,away_team,team,scorer,minute,own_goal,penalty,match_key,data_quality_flag
